#Import And Install

In [ ]:
pip freeze

absl-py==1.4.0
accelerate==1.1.1
aiohappyeyeballs==2.4.3
aiohttp==3.9.5
aiosignal==1.3.1
alabaster==1.0.0
albucore==0.0.19
albumentations==1.4.20
altair==4.2.2
annotated-types==0.7.0
anyio==3.7.1
argon2-cffi==23.1.0
argon2-cffi-bindings==21.2.0
array_record==0.5.1
arviz==0.20.0
astropy==6.1.6
astropy-iers-data==0.2024.11.18.0.35.2
astunparse==1.6.3
async-timeout==4.0.3
atpublic==4.1.0
attrs==24.2.0
audioread==3.0.1
autograd==1.7.0
babel==2.16.0
backcall==0.2.0
beautifulsoup4==4.12.3
bigframes==1.27.0
bigquery-magics==0.4.0
bleach==6.2.0
blinker==1.9.0
blis==0.7.11
blosc2==2.7.1
bokeh==3.6.1
Bottleneck==1.4.2
bqplot==0.12.43
branca==0.8.0
CacheControl==0.14.1
cachetools==5.5.0
catalogue==2.0.10
certifi==2024.8.30
cffi==1.17.1
chardet==5.2.0
charset-normalizer==3.4.0
chex==0.1.87
clarabel==0.9.0
click==8.1.7
cloudpathlib==0.20.0
cloudpickle==3.1.0
cmake==3.30.5
cmdstanpy==1.2.4
colorcet==3.1.0
colorlover==0.3.0
colour==0.1.5
community==1.0.0b1
confection==0.1.5
cons==0.4.6
contourpy==1.3

In [ ]:
pip install requests==2.32.3 ctransformers==0.2.27 sentence-transformers==3.2.1 pinecone-client==5.0.1 langchain==0.3.7
pip install flask==3.0.3 pypdf==5.1.0 huggingface_hub==0.26.2 langchain-pinecone==0.2.0 pinecone-notebooks==0.1.1 langchain-huggingface==0.1.2 langchain_community==0.3.7

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 75.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 81.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 4.2 MB/s eta 0:00:00
  Attempting uninstall: SQLAlchemy
    Found existing installation: SQLAlchemy 2.0.36
    Uninstalling SQLAlchemy-2.0.36:
      Successfully uninstalled SQLAlchemy-2.0.36


In [ ]:
from langchain import PromptTemplate
from langchain.chains import RetrievalQA
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.prompts import PromptTemplate
from langchain.llms import CTransformers

from pinecone import Pinecone

# from langchain_huggingface import HuggingFaceEmbeddings
from langchain_pinecone import PineconeVectorStore
from langchain_core.documents import Document

from uuid import uuid4

#Utility Functions

In [ ]:
import requests

def download_pdf(pdf_url, target_path):
    try:
        # Send a GET request to the PDF URL
        response = requests.get(pdf_url)

        # Raise an error if the request was unsuccessful
        response.raise_for_status()

        # Open the target file in binary write mode and save the content
        with open(target_path, 'wb') as pdf_file:
            pdf_file.write(response.content)

        print(f"PDF downloaded and saved to: {target_path}")

    except requests.exceptions.RequestException as e:
        print(f"Error downloading the PDF: {e}")

In [ ]:
# Extract data from PDF
def load_pdf(data):
  loader = DirectoryLoader(data,
                  glob="*.pdf",
                  loader_cls=PyPDFLoader)
  documents = loader.load()

  return documents

In [ ]:
def text_split(extracted_data):
  text_splitter = RecursiveCharacterTextSplitter(chunk_size = 500, chunk_overlap = 20)
  text_chunks = text_splitter.split_documents(extracted_data)

  return text_chunks

# Data Retrieval
The Gale Encyclopedia of Medicine

In [ ]:
pdf_url = "https://staibabussalamsula.ac.id/wp-content/uploads/2024/06/The-Gale-Encyclopedia-of-Medicine-3rd-Edition-staibabussalamsula.ac_.id_.pdf"
data_folder_path = "./data"
data_path = data_folder_path + "/medicine_books.pdf"

!mkdir $data_folder_path
download_pdf(pdf_url, data_path)

PDF downloaded and saved to: ./data/medicine_books.pdf


# Download The Model
- LLM Model => llama-2-7b-chat.gglmv3.q4_0.bin
- Embedding Model => sentence-transformers/all-MiniLM-L6-v2

In [ ]:
!wget https://huggingface.co/TheBloke/Llama-2-7B-Chat-GGML/resolve/main/llama-2-7b-chat.ggmlv3.q4_0.bin?download=true -O llama-2-7b-chat.ggmlv3.q4_0.bin

--2024-11-21 09:24:03--  https://huggingface.co/TheBloke/Llama-2-7B-Chat-GGML/resolve/main/llama-2-7b-chat.ggmlv3.q4_0.bin?download=true
Resolving huggingface.co (huggingface.co)... 3.167.112.38, 3.167.112.45, 3.167.112.25, ...
Connecting to huggingface.co (huggingface.co)|3.167.112.38|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://cdn-lfs.hf.co/repos/30/e3/30e3aca7233f7337633262ff6d59dd98559ecd8982e7419b39752c8d0daae1ca/8daa9615cce30c259a9555b1cc250d461d1bc69980a274b44d7eda0be78076d8?response-content-disposition=attachment%3B+filename*%3DUTF-8%27%27llama-2-7b-chat.ggmlv3.q4_0.bin%3B+filename%3D%22llama-2-7b-chat.ggmlv3.q4_0.bin%22%3B&response-content-type=application%2Foctet-stream&Expires=1732440243&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTczMjQ0MDI0M319LCJSZXNvdXJjZSI6Imh0dHBzOi8vY2RuLWxmcy5oZi5jby9yZXBvcy8zMC9lMy8zMGUzYWNhNzIzM2Y3MzM3NjMzMjYyZmY2ZDU5ZGQ5ODU1OWVjZDg5ODJlNzQxOWIzOTc1MmM4ZDBkYWFlMW

In [ ]:
def download_hugging_face_embeddings():
  embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
  return embeddings

embeddings = download_hugging_face_embeddings()

<ipython-input-15-458ac3934f71>:2: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or d

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

# API KEY

In [ ]:
PINECONE_API_KEY = "ENTER YOUR PINECONE HERE"
PINECONE_API_HOST = "ENTER YOUR PINECONE HOST HERE"

# Code

In [ ]:
extracted_data = load_pdf(data_folder_path)
len(extracted_data)

4505

In [ ]:
text_chunks = text_split(extracted_data)
len(text_chunks)

39994

In [ ]:
index_name = "medical-chatbot"
pc = Pinecone(api_key=PINECONE_API_KEY)
index = pc.Index(index_name, host=PINECONE_API_HOST)

vector_store = PineconeVectorStore(index=index, embedding=embeddings)

In [ ]:
documents = [Document(page_content=t.page_content) for t in text_chunks]
uuids = [str(uuid4()) for _ in range(len(documents))]

# vector_store.add_documents(documents=documents, ids=uuids)

In [ ]:
query = "What are Allergies?"

docs = vector_store.similarity_search(query, k=3)

print("Result:", docs)

Result: [Document(id='45ece9d0-5e4b-406d-8fb8-61410feb22dd', metadata={}, page_content='‘‘What’s New in: Asthma and Allergic Rhinitis.’’Pulse\n(September 20, 2004): 50.\nRichard Robinson\nTeresa G. Odle\nAllergies\nDefinition\nAllergies are abnormal reactions of the immune\nsystem that occur in response to otherwise harmless\nsubstances.\nDescription\nAllergies are among the most common of medical\ndisorders. It is estimated that 60 million Americans, or\nmore than one in every five people, suffer from some\nform of allergy, with similar proportions throughout'), Document(id='e98d2bd4-b76b-4304-a39a-4fc5c3088b59', metadata={}, page_content='triggered by harmless, everyday substances. This is\nthe condition known as allergy, and the offending\nsubstance is called an allergen. Common inhaled\nallergens include pollen,dust, and insect parts from\ntiny house mites. Common food allergens include\nnuts, fish, and milk.\nAllergic reactions involve a special set of cells\nin the immune system 

In [ ]:
prompt_template = """
Use the following pieces of information to answer the user's question.
If you don't know the answer, just say that you don't know, don't try to make up an answer.

Context: {context}
Question: {question}

Only return the helpful answer below and nothing else.
Helpful answer:
"""

In [ ]:
PROMPT=PromptTemplate(template=prompt_template, input_variables=["context", "question"])
chain_type_kwargs={"prompt": PROMPT}

NameError: name 'PromptTemplate' is not defined

In [ ]:
llm=CTransformers(model="llama-2-7b-chat.ggmlv3.q4_0.bin",
                  model_type="llama",
                  config={"max_new_tokens":512,
                          "temperature":0.8})

In [ ]:
qa=RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=vector_store.as_retriever(search_kwargs={"k": 2}),
    return_source_documents=True,
    chain_type_kwargs=chain_type_kwargs
)

In [ ]:
while True:
  user_input=input(f"Input Prompt:")
  if user_input == "end":
    break
  result=qa({"query": user_input})
  print("Response: ", result["result"])

Input Prompt:how to reduce diabetes type 2?


Response:  The American Diabetes Association recommends that people with diabetes should aim for at least 150 minutes of moderate-intensity aerobic exercise, or 75 minutes of vigorous-intensity aerobic exercise, or an equivalent combination of both, per week. Additionally, they suggest incorporating strength training exercises into one's routine at least twice a week. These exercises can help improve insulin sensitivity and reduce the risk of complications associated with diabetes.
It is important to note that these recommendations are for people with Type II diabetes, not those with other types of diabetes. If you have been diagnosed with diabetes, it is best to consult with your healthcare provider for personalized advice on managing.
Input Prompt:end
